In [1]:
import sys
import asyncio
import time
import os

import numpy as np

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS

In [2]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [3]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

file:///home/patrickingraham/repos/ts_ddsconfig/config/ospl-sp.xml
summit


In [4]:
#get classes and start them
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

atmcs: Adding all resources.
atptg: Adding all resources.
ataos: Adding all resources.
atpneumatics: Adding all resources.
athexapod: Adding all resources.
atdome: Adding all resources.
atdometrajectory: Adding all resources.
atcamera: Adding all resources.
atspectrograph: Adding all resources.
atheaderservice: Adding all resources.
atarchiver: Adding all resources.
Read historical data in 0.05 sec
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, algorithm)
Read 5 history items for RemoteEvent(ATDomeTrajectory, 0, appliedSettingsMatchStart)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, authList)
Read 100 history items for RemoteEvent(ATDomeTrajectory, 0, heartbeat)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, logLevel)
Read 16 history items for RemoteEvent(ATDomeTrajectory, 0, logMessage)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, settingVersions)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, settingsApplied)
Read 1 history i

[[None, None, None, None, None, None, None], [None, None, None, None]]

logMessage DDS read queue is filling: 62 of 100 elements


In [ ]:
tmp = atcs.rem.atptg.evt_summaryState.get()
print(salobj.State(tmp.summaryState))

In [ ]:
tmp = atcs.rem.atmcs.evt_heartbeat.get()
print(tmp)

In [ ]:
tmp =latiss.rem.atspectrograph.evt_settingsApplied.get()
print(tmp)

In [ ]:
await latiss.enable()

In [ ]:
tmp = latiss.rem.atarchiver.evt_heartbeat.get()
print(tmp)

In [ ]:
tmp = latiss.rem.atarchiver.evt_summaryState.get()
print(salobj.State(tmp.summaryState))

In [ ]:
# enable components
await atcs.enable()
#await latiss.standby()

In [ ]:
# ATDOME only
#tmp = await salobj.set_summary_state(atcs.rem.atdome, salobj.State.ENABLED, settingsToApply='test')
tmp = await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.STANDBY)
#tmp=await atcs.rem.atpneumatics.cmd_standby.start()
#print(tmp)

In [ ]:
# ATMCS
tmp = await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.STANDBY)
#tmp=await atcs.rem.atmcs.cmd_start.start(timeout=5)

In [ ]:
# ATPneumatics
tmp = await salobj.set_summary_state(atcs.rem.atpneumatics, salobj.State.ENABLED)
#tmp=await atcs.rem.atpneumatics.cmd_enable.start()

In [ ]:
# ATPtg
# tmp = await salobj.set_summary_state(atcs.rem.atptg, salobj.State.ENABLED)
#tmp=await atcs.rem.atptg.cmd_enable.start()
# Need to manually say which rotator we're using at the moment (N2-->focus=3, N1-->focus=2)
await atcs.rem.atptg.cmd_focusName.set_start(focus=3)

In [ ]:
# ATAOS
tmp = await salobj.set_summary_state(atcs.rem.ataos, salobj.State.ENABLED, settingsToApply='hex_m1_hex_202003_v3.yaml')
#tmp=await atcs.rem.ataos.cmd_enable.start()

In [ ]:
#await atcs.rem.atdome.cmd_homeAzimuth.start()

In [ ]:
#mp = await salobj.set_summary_state(atcs.rem.athexapod, salobj.State.ENABLED, settingsToApply='summit')
#tmp=await atcs.rem.athexapod.cmd_start.set_start(settingsToApply='summit')
tmp=await atcs.rem.athexapod.cmd_enable.start()
print(tmp)

In [ ]:
# ATSPECTROGRAPH
tmp = await salobj.set_summary_state(latiss.rem.atspectrograph, salobj.State.ENABLED, settingsToApply='LATISS_AuxTel_20210108_v3.yaml')
#tmp=await latiss.rem.atspectrograph.cmd_start.set_start(settingsToApply='LATISS_AuxTel_20210108.yaml')
#tmp=await latiss.rem.atspectrograph.cmd_enable.start()
print(tmp)

In [ ]:
# take event checking out the slew commands to test telescope only
atcs.check.atdome = False
atcs.check.atdometrajectory = False

In [ ]:
offset = 1.12 #2./3600
start_az=205+offset
start_el=71+offset
start_rot_pa=1.5+offset
await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=False)

In [ ]:
# Perform Rotator verification (LTS-337-003)True

In [ ]:
await atcs.rem.atmcs.cmd_abort.start()

In [ ]:
await atcs.stop_tracking()

In [ ]:
start_az=-58
start_el=35
start_rot_pa=13.1
await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=False)

In [ ]:
start_az=165.5
start_el=67.9
start_rot_pa=170

start_time =time.time()
await atcs.point_azel(start_az, start_el, rot_pa=start_rot_pa, wait_dome=True)
end_time = time.time()
print(f'Time to perform {170*2} degree rotation of N2 is {end_time-start_time} seconds')

In [ ]:
# send telescope to alt/az/
#await salobj.set_summary_state(latiss.atcamera, salobj.State.ENABLED)

In [ ]:
# point telescope to desired starting position
# this make take extra time as more checks are performed
start_az=200
start_el=71
start_rot_pa=0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=True)

In [ ]:
#declare offset size
d_az= 3.5 # degrees
d_el = 3.5 # degrees
d_rot= 3.5 # degrees

d_az= 10 # degrees
d_el = 10 # degrees
d_rot= 10 # degrees

d_az= 90 # degrees
d_el = 60 # degrees
d_rot= 90 # degrees

In [ ]:
await atcs.stop_tracking()

In [ ]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")

In [ ]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")

# Perform tracking test by taking continous 20s exposures
#await attcs.slew_object('HD 167060')
#await atcs.slew_object('HIP 100787')
await atcs.slew_object('51 Cap')

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")

In [ ]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")
await atcs.stop_tracking()

In [ ]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")
await atcs.stop_tracking()

In [ ]:
await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')

In [ ]:
await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')

In [ ]:
while True:
    await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')
    await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')

In [ ]:
# stopped tracking elsewhere, then reslewed
await atcs.slew_object('HR 7474')
#await atcs.slew_object('HIP 23172')

In [ ]:
#await atcs.rem.ataos.cmd_setLogLevel.set_start(level=10)

In [ ]:
# OFFSET  AZ/EL
await attcs.offset_azel(az=-5*60*60, el=0)
await attcs.offset_azel(az=-2*60*60, el=0)
await attcs.offset_azel(az=0, el=0)

In [ ]:
while True:
    await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')
    await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')
    

In [ ]:
await latiss.take_bias(nbias=1)

In [ ]:
#await attcs.startup()

In [ ]:
await latiss.rem.atarchiver.cmd_start.start()

In [ ]:
# turn on corrections
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# turn off corrections
tmp = await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# is ATAOS correcting?
tmp = atcs.rem.ataos.evt_correctionEnabled.get()
print(tmp)

In [ ]:
# what is the M1 mirror pressure
pres = atcs.rem.ataos.evt_m1CorrectionStarted.get()
print(pres)

In [ ]:
# Verify that ATAOS handles focus/pointing offsets from grating change
await latiss.rem.atspectrograph.cmd_changeFilter.set_start(filter=1)
await latiss.rem.atspectrograph.cmd_changeDisperser.set_start(disperser=1)

In [ ]:
await latiss.setup_atspec

In [ ]:
# change ataos wavelength
tmp = await atcs.rem.ataos.cmd_setWavelength.set_start(wavelength=700)
print(tmp)

In [ ]:
# get offsets from ATAOS
focus_offsets = atcs.rem.ataos.evt_focusOffsetSummary.get()
print(focus_offsets)

In [ ]:
# get offsets from ATAOS
focus_offsets = await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(focus_offsets)

In [ ]:
# get pointing offsets from ATAOS
pointing_offsets = await atcs.rem.ataos.evt_pointingOffsetSummary.aget()
print(pointing_offsets)

In [ ]:
# get pointing offsets from ATAOS
pointing_offsets = atcs.rem.ataos.evt_pointingOffsetSummary.get()
print(pointing_offsets)

In [ ]:
# reset offsets
tmp = await atcs.rem.ataos.cmd_resetOffset.set_start(axis='z')

In [ ]:
# take LATISS images
await latiss.take_bias(nbias=30)